<a href="https://colab.research.google.com/github/kadirdundar/Challenge_of_Juniper_notebooks/blob/main/Starter_Notebook_Juniper_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Objective:
A Retail Superstore chain wants to gain insights on the performance of each its stores and make strategic decisions on its operating model.

As a Data Scientist you are required to understand the data provided and help the client predict the sales at the register in current hour based on the sales attribute associated with each store.

The given data consists of 4 broad attribute types:
   * Register: Attributes related to the register at which the sales transaction has taken place
   * Cashier: Attributes related to cashier working at the register
   * Store: Attributes related to a particular store
   * Region: Attributes related to a group of stores in the region



## Problem Statement:
Predict the sales at the register in current hour and prioritize the individual attributes that affect the sales.


#### Upload your datasets to colab using upload to storage session taskbar

## Import Required Libraries

In [122]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import colors
#
import xgboost as xgb
#
import os
import sys
#
from datetime import datetime, timedelta
from time import time
from uuid import uuid4
from scipy.ndimage import convolve1d
#
from sklearn.metrics import make_scorer, r2_score
from sklearn.model_selection import cross_validate
# 
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)



%matplotlib inline

## Label Encoding and Null Value Imputaion Functions

In [123]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [124]:
df = pd.read_csv("/content/drive/MyDrive/yarisma/training_dataset.csv")


In [125]:
def checkDf(dataframe, head = 8):
  print("##### Shape #####")
  print(dataframe.shape)

  print("\n##### Null Analysis #####")
  print(dataframe.isnull().sum())



checkDf(df)
df.head()  

##### Shape #####
(2058, 63)

##### Null Analysis #####
Unnamed: 0                                       0
observation_id                                   0
observation_timestamp                            0
hour_of_day                                      0
register__sales_dollar_amt_this_hour             0
                                              ... 
region__nighttime_returns_amt_per_hour           1
region__peak_sales_dollar_amt_per_hour           1
region__peak_sales_dollar_amt_per_hour_v2       17
region__peak_returns_dollar_amt_per_hour         1
region__peak_returns_dollar_amt_per_hour_v2    221
Length: 63, dtype: int64


,Unnamed: 0,observation_id,observation_timestamp,hour_of_day,register__sales_dollar_amt_this_hour,register__payment_types_accepted,register__peak_sales_dollar_amt_per_hour,register__sales_dollar_amt_last_hour,register__sales_quantity_last_hour,register__sales_quantity_rescanned_frac,...,region__sales_dollar_amt_last_hour,region__returns_dollar_amt_last_hour,region__nighttime_open_registers,region__nighttime_service_time_per_customer,region__nighttime_sales_amt_per_hour,region__nighttime_returns_amt_per_hour,region__peak_sales_dollar_amt_per_hour,region__peak_sales_dollar_amt_per_hour_v2,region__peak_returns_dollar_amt_per_hour,region__peak_returns_dollar_amt_per_hour_v2
0,0,704d2a80-d52e-11ec-90ff-c7e6292284b3,2022-05-16 15:39:57,15,347.29,Cash+Credit,-0.7383,-0.1270,-0.1993,-0.8299,...,-0.6920,-0.4605,-0.5180,-1.0062,-0.6462,-0.6030,-0.4773,0.1748,-1.7951,-0.8284
1,1,1cacc1d0-e6ac-11ec-b65d-156af70ce36b,2022-06-07 21:52:23,21,361.59,Cash+Credit,0.6483,-0.0362,-0.0777,-0.7395,...,-0.6531,-0.4434,-0.6498,0.9031,-0.6493,-0.6106,0.4998,-0.9816,0.8939,-0.8614
2,2,6dc2b330-d37a-11ec-884e-dfe9ea4a7bd5,2022-05-14 11:38:52,11,850.73,Cash+Credit,-0.4950,-0.1268,-0.1974,1.3139,...,-0.6120,-0.1786,-0.6040,-1.4229,-0.6456,-0.6037,1.9337,-0.9093,2.4046,-0.7567
3,3,163ee0a0-0cca-11ed-a73c-8904b24187cc,2022-07-26 10:02:41,10,1175.69,Cash+Credit,-0.5594,-0.1270,-0.1991,-0.8299,...,-0.5472,0.1246,-0.5925,-1.4097,-0.6478,-0.6079,0.1288,-0.9093,-0.4983,-0.7567
4,4,5e3c5df0-d5ee-11ec-a5f2-3b6f99e95850,2022-05-17 14:33:50,14,3204.53,Cash+Credit,0.5693,-0.1221,-0.1632,-0.7071,...,-0.3838,0.1996,-0.5696,1.1231,-0.6475,-0.6032,1.0050,0.1748,1.5660,0.2356


In [126]:
def grabColNames(dataframe, catTh=10, carTh=20):
    """

    Veri setindeki kategorik, numerik ve kategorik fakat kardinal değişkenlerin isimlerini verir.
    Not: Kategorik değişkenlerin içerisine numerik görünümlü kategorik değişkenler de dahildir.

    Parameters
    ------
        dataframe: dataframe
                Değişken isimleri alınmak istenilen dataframe
        catTh: int, optional
                numerik fakat kategorik olan değişkenler için sınıf eşik değeri
        carTh: int, optinal
                kategorik fakat kardinal değişkenler için sınıf eşik değeri

    Returns
    ------
        catCols: list
                Kategorik değişken listesi
        numCols: list
                Numerik değişken listesi
        catButCar: list
                Kategorik görünümlü kardinal değişken listesi

    Examples
    ------
        import seaborn as sns
        df = sns.load_dataset("iris")
        print(grabColNames(df))


    Notes
    ------
        catCols + numCols + catButCar = toplam değişken sayısı
        numButCat catCols'un içerisinde.
        Return olan 3 liste toplamı toplam değişken sayısına eşittir: catCols + numCols + catButCar = değişken sayısı

    """

    # catCols, catButCar
    catCols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    numButCat = [col for col in dataframe.columns if dataframe[col].nunique() < catTh and
                 dataframe[col].dtypes != "O"]
    catButCar = [col for col in dataframe.columns if dataframe[col].nunique() > carTh and
                 dataframe[col].dtypes == "O"]
    catCols = catCols + numButCat
    catCols = [col for col in catCols if col not in catButCar]

    # numCols
    numCols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    numCols = [col for col in numCols if col not in numButCat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'catCols: {len(catCols)}')
    print(f'numCols: {len(numCols)}')
    print(f'catButCar: {len(catButCar)}')
    print(f'numButCat: {len(numButCat)}')
    return catCols, numCols, catButCar


catCols, numCols, catButCar = grabColNames(df)

Observations: 2058
Variables: 63
catCols: 10
numCols: 51
catButCar: 2
numButCat: 8


In [127]:
catCols

['register__payment_types_accepted',
 'store__type_code',
 'cashier__title_level',
 'cashier__n_years_experience',
 'store__n_employees_total',
 'store__n_managers',
 'store__n_baggers',
 'store__is_sufficiently_staffed',
 'region__peak_sales_dollar_amt_per_hour_v2',
 'region__peak_returns_dollar_amt_per_hour_v2']

In [128]:
numCols

['Unnamed: 0',
 'hour_of_day',
 'register__sales_dollar_amt_this_hour',
 'register__peak_sales_dollar_amt_per_hour',
 'register__sales_dollar_amt_last_hour',
 'register__sales_quantity_last_hour',
 'register__sales_quantity_rescanned_frac',
 'register__sales_payments_declined_frac',
 'register__peak_returns_dollar_amt_per_hour',
 'register__returns_dollar_amt_last_hour',
 'register__returns_quantity_last_hour',
 'register__returns_quantity_rescanned_frac',
 'cashier__hours_into_shift',
 'cashier__item_scan_rate_per_min',
 'cashier__item_manual_entry_rate_per_min',
 'store__miles_to_nearest_location',
 'store__target_sales_quantity_per_hour',
 'store__mean_customer_to_staff_ratio',
 'store__mean_service_time_per_customer',
 'store__n_open_registers',
 'store__occupancy_main_floor',
 'store__occupancy_grocery',
 'store__occupancy_checkout_areas',
 'store__occupancy_food_court',
 'store__occupancy_backrooms',
 'store__occupancy_indoors',
 'store__occupancy_outdoors',
 'store__outdoor_temp

In [129]:
catButCar

['observation_id', 'observation_timestamp']

In [130]:
naCols = [col for col in df.columns if df[col].isnull().sum() > 0]
print(naCols)

['store__gift_returns_quantity_last_hour', 'region__n_stores', 'region__n_open_registers', 'region__mean_service_time_per_customer', 'region__stdev_service_time_per_customer', 'region__sales_dollar_amt_last_hour', 'region__returns_dollar_amt_last_hour', 'region__nighttime_open_registers', 'region__nighttime_service_time_per_customer', 'region__nighttime_sales_amt_per_hour', 'region__nighttime_returns_amt_per_hour', 'region__peak_sales_dollar_amt_per_hour', 'region__peak_sales_dollar_amt_per_hour_v2', 'region__peak_returns_dollar_amt_per_hour', 'region__peak_returns_dollar_amt_per_hour_v2']


In [131]:
a = df["region__n_stores"].mean()
df["region__n_stores"].fillna(f"{a}", inplace = True)


In [132]:
naCols = [col for col in df.columns if df[col].isnull().sum() > 0]
naCols



['store__gift_returns_quantity_last_hour',
 'region__n_open_registers',
 'region__mean_service_time_per_customer',
 'region__stdev_service_time_per_customer',
 'region__sales_dollar_amt_last_hour',
 'region__returns_dollar_amt_last_hour',
 'region__nighttime_open_registers',
 'region__nighttime_service_time_per_customer',
 'region__nighttime_sales_amt_per_hour',
 'region__nighttime_returns_amt_per_hour',
 'region__peak_sales_dollar_amt_per_hour',
 'region__peak_sales_dollar_amt_per_hour_v2',
 'region__peak_returns_dollar_amt_per_hour',
 'region__peak_returns_dollar_amt_per_hour_v2']

In [133]:
for cols in naCols:
  for col in numCols:
    if cols == col:
      a = df[col].mean()
      df[col].fillna(f"{a}", inplace = True)
      
  for col in catCols:
    if cols==col :
       a = df[col].median()
       df[col].fillna(f"{a}", inplace = True)




In [134]:
naCols = [col for col in df.columns if df[col].isnull().sum() > 0]
naCols


[]

**ENCODİNG**

In [135]:
from sklearn.preprocessing import  LabelEncoder

In [136]:
df = pd.get_dummies(df, columns=["register__payment_types_accepted"],drop_first=True)

df = pd.get_dummies(df, columns=['store__type_code'],drop_first=True)






        



  
    


In [137]:
df = pd.get_dummies(df, columns=['cashier__title_level'])
df = pd.get_dummies(df, columns=['cashier__n_years_experience'])
df = pd.get_dummies(df, columns=['store__n_employees_total'])
df = pd.get_dummies(df, columns=['store__n_managers'])
df = pd.get_dummies(df, columns=['store__n_baggers'])
df = pd.get_dummies(df, columns=['store__is_sufficiently_staffed'])
df = pd.get_dummies(df, columns=['region__peak_sales_dollar_amt_per_hour_v2'])
df = pd.get_dummies(df, columns=['region__peak_returns_dollar_amt_per_hour_v2'])

In [138]:
df.head()

,Unnamed: 0,observation_id,observation_timestamp,hour_of_day,register__sales_dollar_amt_this_hour,register__peak_sales_dollar_amt_per_hour,register__sales_dollar_amt_last_hour,register__sales_quantity_last_hour,register__sales_quantity_rescanned_frac,register__sales_payments_declined_frac,...,region__peak_returns_dollar_amt_per_hour_v2_-0.8614,region__peak_returns_dollar_amt_per_hour_v2_-0.8559,region__peak_returns_dollar_amt_per_hour_v2_-0.8449,region__peak_returns_dollar_amt_per_hour_v2_-0.8394,region__peak_returns_dollar_amt_per_hour_v2_-0.8339,region__peak_returns_dollar_amt_per_hour_v2_-0.8284,region__peak_returns_dollar_amt_per_hour_v2_-0.7567,region__peak_returns_dollar_amt_per_hour_v2_0.2356,region__peak_returns_dollar_amt_per_hour_v2_1.3382,region__peak_returns_dollar_amt_per_hour_v2_-0.7567
0,0,704d2a80-d52e-11ec-90ff-c7e6292284b3,2022-05-16 15:39:57,15,347.29,-0.7383,-0.1270,-0.1993,-0.8299,-0.1247,...,0,0,0,0,0,1,0,0,0,0
1,1,1cacc1d0-e6ac-11ec-b65d-156af70ce36b,2022-06-07 21:52:23,21,361.59,0.6483,-0.0362,-0.0777,-0.7395,-0.1135,...,1,0,0,0,0,0,0,0,0,0
2,2,6dc2b330-d37a-11ec-884e-dfe9ea4a7bd5,2022-05-14 11:38:52,11,850.73,-0.4950,-0.1268,-0.1974,1.3139,0.1075,...,0,0,0,0,0,0,1,0,0,0
3,3,163ee0a0-0cca-11ed-a73c-8904b24187cc,2022-07-26 10:02:41,10,1175.69,-0.5594,-0.1270,-0.1991,-0.8299,-0.1247,...,0,0,0,0,0,0,1,0,0,0
4,4,5e3c5df0-d5ee-11ec-a5f2-3b6f99e95850,2022-05-17 14:33:50,14,3204.53,0.5693,-0.1221,-0.1632,-0.7071,-0.1247,...,0,0,0,0,0,0,0,1,0,0


In [139]:
df = df.drop(["observation_id"], axis=1)


In [156]:
df = df.drop(["observation_timestamp"], axis=1)


In [157]:
y = df.drop(["register__sales_dollar_amt_this_hour"], axis=1)
X = df["register__sales_dollar_amt_this_hour"]

In [158]:
from sklearn.preprocessing import  RobustScaler

In [159]:
rs = RobustScaler()
df = rs.fit_transform(df)

In [160]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
import xgboost
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [171]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=17)

m = XGBRegressor(objective='reg:squarederror').fit(X_train, y_train)

#pred = m.predict(X_test)
##rmse = np.sqrt(MSE(y_test, pred))
#print(f"RMSE: {round(rmse, 4)}  ")


ValueError: ignored

## Weight Sampling to reduce bias

In [140]:
def get_smoothing_kernel(kernel_size=5, sigma=1.0):
    '''Builds a Gaussian smoothing kernel of a desired size (# of elements) and
       spread. The Gaussian will be positioned at the center of the kernel.'''

    # Calculate a Gaussian with mean x0 = 0 and std = sigma
    # NOTE: Renormalize so sum(y) = 1.0
    x = np.linspace(-(kernel_size-1) / 2., (kernel_size - 1) / 2., kernel_size)   # x-lims for Gaussian
    kernel = np.exp(-0.5 * np.square(x) / np.square(sigma))
    kernel = kernel / np.sum(kernel)

    return kernel


def get_sample_weights(target_vals, target_bins,
                       smoothing_kernel_props={'size': 51, 'sigma': 5},
                       weighting_method='inverse_square',
                       max_weight=100,
                       make_plots=False):
    '''Gets reweighting factors for each sample in a regression problem, following the
       LDS method detailed here:
           https://towardsdatascience.com/strategies-and-tactics-for-regression-on-imbalanced-data-61eeb0921fca

       Essentially, we do these steps:
       1.) Get a histogram of target values
       2.) Use a Gaussian filter to smooth the histogram
       3.) Use inverse or inverse-sqrt of counts to weight each bin
       4.) Map each sample to its new weight

       The output 'sample_weights' vector will be the same shape as 'target_vals'.
    '''

    # Get the histogram / distribution of target values -- these will be our 'classes'
    # NOTE: Convert to float for convolution below
    [counts, bins] = np.histogram(target_vals, bins=target_bins)
    counts = [float(c) for c in counts]

    # Apply a Gaussian smoothing filter to the counts distribution
    # NOTE: Renormalize so the convolved distribution maintains the same # of total counts
    kernel = get_smoothing_kernel(kernel_size=smoothing_kernel_props['size'], sigma=smoothing_kernel_props['sigma'])
    smooth_counts = convolve1d(counts, weights=kernel, mode='mirror')
    smooth_counts = smooth_counts * ( sum(counts) / sum(smooth_counts) )

    # Make sure there are no 0-count bins. Replace them with the lowest non-zero count
    mask = ( smooth_counts == 0 )
    smooth_counts[mask] = np.min( smooth_counts[np.nonzero(smooth_counts)] )

    # Get weights for each 'class' from the smoothed counts
    # NOTE: Supports either 1/N or 1/N^2 reweighting
    class_weights = np.ones(smooth_counts.shape)   # Default: No reweighting
    if ( weighting_method == 'inverse_square' ):
        class_weights = 1 / np.sqrt(smooth_counts)
    if ( weighting_method == 'inverse' ):
        class_weights = 1 / smooth_counts

    # Now map each sample to the corresponding class weight
    # NOTE: np.digitize returns the bin index used in histogramming. With k bins, we get k-1 class weights
    idxs = np.digitize(target_vals, target_bins) - 1
    sample_weights = class_weights[idxs]

    # Rescale weights so the sum(sample_weights) = n_samples. This is what we'd get without reweighting
    n_samples = len(target_vals)
    scaling_factor = n_samples / sum(sample_weights)
    sample_weights = sample_weights * scaling_factor

    # Truncate very large weights to reduce noise (especially helpful for 'inverse' weighting mode)
    sample_weights = np.clip(sample_weights, 0, max_weight)

    # (OPTIONAL) Make plots of both the weights & the target distributions (smoothed & unsmoothed)
    if ( make_plots ):

        # Plot 1: Target distribution (smoothed vs. unsmoothed)
        fig = plt.figure(figsize=[8,4])
        ax = fig.gca()
        plt.plot(target_bins[1:], counts)
        plt.plot(target_bins[1:], smooth_counts, color='orange')
        plt.title('Class Distributions', size=20)
        plt.xlabel('Target Value', size=16)
        plt.ylabel('Counts', size=16)
        ax.tick_params(axis='both', labelsize=14)
        plt.xlim([target_bins[0], target_bins[-1]])
        plt.ylim([0,None])
        plt.grid('minor')
        plt.legend(['Original', 'Smoothed'], fontsize=14)

        # Plot 2: Class weights
        fig = plt.figure(figsize=[8,4])
        ax = fig.gca()
        plt.plot(target_bins[1:], np.clip(class_weights * scaling_factor, 0, max_weight))
        plt.hlines([1], target_bins[0], target_bins[-1], colors='orange')
        plt.title('Class Weights', size=20)
        plt.xlabel('Target Value', size=16)
        plt.ylabel('Weight', size=16)
        ax.tick_params(axis='both', labelsize=14)
        plt.xlim([target_bins[0], target_bins[-1]])
        plt.ylim([0,None])
        plt.grid('minor')
        plt.legend(['Reweighted', 'Unweighted'], fontsize=14)

    return sample_weights


## Define Hyperparameter Tuning parameters

In [141]:
HYPERPARAMETER_DTYPES = {
    'n_estimators': int,
    'max_depth': int,
    'learning_rate': float
}

## Define Heperparameter tuning function

In [142]:
def random_search_xgb(hyperparameter_space, X_train, y_train,
                      wt_train=None,
                      metric_to_optimize='r2',
                      n_runs=100,
                      n_folds=5,
                      n_jobs=4,
                      print_status=True):
    '''Performs random search hyperparameter tuning for an XGBoost model. We'll evaluate model
       performance at many random points in the hyperparameter space, choosing the best result
       that was found. Supports sample reweighting via 'wt_train'.

       Parameters are optimized using cross-validation of a specified model evaluation metric:
           -- 'r2'
           -- 'rmse'
           -- 'mae'
           -- 'mape'
           -- 'racc'
       This last one is the 'regression accuracy' at 10% maximum error, i.e. the number
       of predictions that are within 10% of true.

       Each key in the hyperparameter_space dictionary should be a named hyperparameter,
       while values should be arrays containing the min/max values to test.

       Returns the trained optimal model, along with performance metrics and the optimal
       hyperparameters used.

       NOTE: Currently only support 'n_estimators', 'max_depth', and 'learning_rate'.'''

    t0 = time()

    # Initialize the return values
    optimal_model = None
    optimal_hyperparameters = {}
    optimal_eval_metrics = None

    # Print status message
    if ( print_status ):
        print('XGBoost Hyperparameter Search')
        print('-'*40)
        print('Method:       {0:s}'.format('random-search'))
        print('Dataset:      {0:d} pts x {1:d} dimensions'.format(X_train.shape[0], X_train.shape[1]))
        print('Optimizer:    {0:s}'.format(metric_to_optimize.upper()))
        print('# of Runs:    {0:d}'.format(n_runs))
        print()
        print('Hyperparameter Space:')
        for param in hyperparameter_space.keys():
            print('-- {0:<16s} [{1:s}]'.format(param+':', ', '.join([str(val) for val in hyperparameter_space[param]]) ))
        print('\n')

    # Try a bunch of random sets of hyperparameters, returning results for the best one
    optimal_metric_val = None
    n_evals = 0
    for run_num in range(n_runs):

        # Pick some random hyperparameters to start with
        current_hyperparameters = get_random_hyperparameters(hyperparameter_space)
        current_metric_val = None

        # (OPTIONAL) Print progress
        if ( print_status ):
            format_str = 'Progress: Run {0:>3s} / {1:d}   -->   Hyperparameters: {2:s}' + ' '*80
            print(format_str.format('#{0:d}'.format(run_num+1), n_runs, str(current_hyperparameters)), end='\r', flush=True)

        # Evaluate an XGBoost Regressor with the current hyperparameters
        current_model = initialize_model_xgb(current_hyperparameters)
        [current_eval_results, _] = evaluate_regression_model(current_model, X_train=X_train, y_train=y_train, wt_train=wt_train,
                                                              n_folds=n_folds, n_jobs=n_jobs)
        n_evals += 1

        # Compare model performance w/ best found so far
        current_metric_val = current_eval_results[metric_to_optimize]['mean']
        if ( ( optimal_metric_val is None ) or ( current_metric_val > optimal_metric_val ) ):
            optimal_metric_val = current_metric_val
            optimal_hyperparameters = current_hyperparameters
            optimal_eval_results = current_eval_results


    # Train the final optimized model
    optimal_model = train_model_xgb(optimal_hyperparameters, X_train, y_train, wt_train=wt_train)

    # (OPTIONAL) Print summary
    if ( print_status ):
        print('\n\n')
        print('Search Complete')
        print('-'*40)
        print('Optimal Params:      {0:s}'.format(str(optimal_hyperparameters)))
        print('Model Performance:   {0:s} = {1:3.4f}'.format(metric_to_optimize, optimal_metric_val))
        print('Params Evaluated:    {0:d}'.format(n_evals))
        print('Execution Time:      {0:3.2f}s'.format(time()-t0))
        print()

    return optimal_model, optimal_hyperparameters, optimal_eval_results


def get_random_hyperparameters(hyperparameter_space):
    '''Randomly selects a point in hyperparameter space. Outputs will fall within the
       provided ranges'''

    # For each hyperparameter, pick a random value somewhere within the provided range
    hyperparameters = {}
    for param in hyperparameter_space.keys():
        min_value = min(hyperparameter_space[param])
        max_value = max(hyperparameter_space[param])
        hyperparameters[param] = min_value + (max_value - min_value) * np.random.random()

        # Convert to desired datatype
        if ( HYPERPARAMETER_DTYPES[param] == int ):
            hyperparameters[param] = int(np.round(hyperparameters[param]))

    return hyperparameters


def initialize_model_xgb(hyperparameters):
    '''Initializes an XGBoost Regressor w/ desired hyperparameters'''

    xgb_model = xgb.XGBRegressor(n_estimators=int(hyperparameters['n_estimators']),
                                 max_depth=int(hyperparameters['max_depth']),
                                 learning_rate=hyperparameters['learning_rate'])

    return xgb_model


def train_model_xgb(hyperparameters, X_train, y_train, wt_train=None):
    '''Trains an XGBoost Regressor with a desired set of hyperparameters. Supports
       sample reweighting via 'wt_train'.'''

    # Initialize the model w/ desired hyperparameters
    xgb_model = initialize_model_xgb(hyperparameters)

    # Fit to training data
    xgb_model.fit(X_train, y_train, sample_weight=wt_train)

    return xgb_model


## Model Evaluation Functions

In [143]:
def evaluate_regression_model(regression_model,
                              X_train=None,
                              y_train=None,
                              wt_train=None,
                              X_test=None,
                              y_test=None,
                              wt_test=None,
                              y_lims=[None,None],
                              n_folds=5,
                              n_jobs=4):
    '''Evaluates a regression model on a set of common performance metrics. Cross-validation
       is used to evaluate the training set (X_train, y_train), resulting in multiple values
       per metric. For the test set (X_test, y_test), metrics are computed on the model's
       predictions, resulting in a single value per metric. Supports any model architecture
       with *.fit() and *.predict() methods.

       Supports sample reweighting via 'wt_train'.

       NOTE: User has the option to coerce predictions within the range given by y_lims.'''

    # Initialize return values
    eval_results__train = {}
    eval_results__test = {}

    # If sample weights are provided, package them as a kwarg for cross_validate()
    fit_params = None
    if ( wt_train is not None ):
        fit_params = {'sample_weight': wt_train}

    # If a training set is provided, run cross-validation on it
    if ( ( X_train is not None ) and ( y_train is not None ) ):

        # Cross-validate model performance
        eval_metrics = {
            'r2': make_scorer(r2_score),
            'rmse': make_scorer(rmse),
            'mae': make_scorer(mae),
            'mape': make_scorer(mape),
            'racc': make_scorer(regression_accuracy)
        }
        eval_results = cross_validate(regression_model, X_train, y_train, fit_params=fit_params,
                                      cv=n_folds, scoring=eval_metrics, n_jobs=n_jobs)

        # Let's compute means & standard dev's for all metrics, and change the output names
        # NOTE: Cross-validation above gives the metric names a 'test_' prefix
        output_name_mappings = {
            'fit_time': 'fit_time',
            'score_time': 'score_time',
            'test_r2': 'r2',
            'test_rmse': 'rmse',#
            'test_mae': 'mae',
            'test_mape': 'mape',
            'test_racc': 'racc'
        }
        for metric in eval_results.keys():
            metric_ = output_name_mappings[metric]
            eval_results__train[metric_] = {
                'mean': np.mean(eval_results[metric]),
                'std': np.std(eval_results[metric]),
                'data': eval_results[metric]
            }

    # If a test set is provided, run evaluation on the predictions
    if ( ( X_test is not None ) and ( y_test is not None ) ):

        # Get measured & predicted results for the test dataset
        y_meas = y_test
        y_pred = regression_model.predict(X_test)

        # (OPTIONAL) Coerce predictions into allowed range (e.x. non-negative)
        if ( y_lims[0] is not None ):
            mask = y_pred < y_lims[0]
            y_pred[mask] = y_lims[0]
        if ( y_lims[1] is not None ):
            mask = y_pred > y_lims[1]
            y_pred[mask] = y_lims[1]

        # Compute the desired performance metrics
        eval_results__test = {
            'data': {'y_meas': y_meas, 'y_pred': y_pred},
            'r2': r2_score(y_meas, y_pred),
            'rmse': rmse(y_meas, y_pred),
            'mae': mae(y_meas, y_pred),
            'mape': mape(y_meas, y_pred),
            'racc': regression_accuracy(y_meas, y_pred)
        }
    return eval_results__train, eval_results__test


def regression_accuracy(y_meas, y_pred, max_error=20, error_type='relative'):
    '''Compares predicted & measured values, returning the percentage of predictions
       that are within a set error limit. This error limit can be an absolute value
       or a relative percentage'''

    # OPTION 1: Relative percentage
    if ( error_type == 'relative' ):
        mask = 100.0 * abs((y_pred - y_meas) / y_meas) < max_error

    # OPTION 2: Absolute value
    if ( error_type == 'absolute' ):
        mask = abs(y_pred - y_meas) < max_error

    accuracy = sum(mask) / len(mask)

    return accuracy


def rmse(y_meas, y_pred):
    '''Computes RMSE for predicted vs. measured data points'''

    n_samples = len(y_meas)
    rmse = np.linalg.norm(y_pred - y_meas) / np.sqrt(n_samples)

    return rmse


def mae(y_meas, y_pred):
    '''Computes MAE for predicted vs. measured data points'''

    n_samples = len(y_meas)
    mae = np.sum(np.abs(y_pred - y_meas)) / n_samples

    return mae


def mape(y_meas, y_pred):
    '''Computes MAPE for predicted vs. measured data points'''

    n_samples = len(y_meas)
    mape = np.sum(np.abs((y_pred - y_meas)/y_meas)) / n_samples

    return mape


## Function to  visualize data

In [144]:
# Define a few standard color palettes
color_palettes = {

    'tableau': [
        'tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple',
        'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan'
    ],

    'blues': ['darkblue', 'blue', 'slateblue', 'rebeccapurple', 'indigo'],

    'oranges': ['darkorange', 'orange', 'sandybrown', 'peru', 'saddlebrown']

}



In [145]:
def plot_distribution(data,
                      data_labels=None,
                      bins=None,
                      title='Distribution',
                      xlabel='Variable',
                      hist_units='counts',
                      output_filepath=None):
    '''Plots the probability distribution of one or more variables. User can choose
       output in counts/bin or probability/bin.'''

    # Define default bin structure if not provided
    if ( bins is None ):
        n_bins = 50
        min_val = None
        max_val = None
        for dataset in data:
            this_min = np.nanmin(dataset)
            this_max = np.nanquantile(dataset, 0.99) #np.nanmax(dataset)
            if ( min_val is None ):
                min_val = this_min
            elif ( this_min < min_val ):
                min_val = this_min
            if ( max_val is None ):
                max_val = this_min
            elif ( this_max > max_val ):
                max_val = this_max
        bins = np.linspace(min_val, max_val, n_bins)

    # Set up plot
    fig = plt.figure(figsize=[10,8], facecolor='w')
    ax = plt.gca()

    # Plot a histogram for each input data series
    # NOTE: Normalize to probability per bin if desired
    for dataset in data:
        [counts,bins] = np.histogram(dataset, bins=bins)
        counts_ = np.append([0], counts)
        if ( hist_units == 'probability' ):
            counts_ = counts_ / sum(counts_)
        plt.step(bins,counts_)

    # Set up plot properties
    plt.title(title, size=24)
    plt.xlabel(xlabel, size=20)
    if ( hist_units == 'probability' ):
        plt.ylabel('Probability Density\n(1/Bin)', size=20)
    if ( hist_units == 'counts' ):
        plt.ylabel('Counts\n(#/Bin)', size=20)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.xlim([bins[0],bins[-1]])
    plt.ylim([0,None])
    plt.grid('minor')

    # Set up legend
    if ( data_labels is not None ):
        ax.legend(data_labels, fontsize=16, shadow=True)
        #bbox_to_anchor = [1.05,0.5]
        #ax.legend(data_labels, loc='center left', fontsize=16, shadow=True, bbox_to_anchor=bbox_to_anchor)

    # (OPTIONAL) Save figure if desired
    if ( output_filepath is not None ):
        plt.savefig(output_filepath, bbox_inches='tight')
    plt.show()
#

def plot_heatmap(x, y,
                 x_bins=None,
                 y_bins=None,
                 aspect='equal',
                 scale='log',
                 title='Heatmap',
                 xlabel='Variable 1',
                 ylabel='Variable 2',
                 cmap='Reds',
                 plot_diagonal=False,
                 output_filepath=None):
    '''Renders a heatmap of y vs. x. Generally produces a clearer visualization
       than a simple scatterplot'''

    # Define default bin structures if they weren't provided
    if ( x_bins is None ):
        x_bins = np.linspace(min(x), max(x), 100)
    if ( y_bins is None ):
        y_bins = np.linspace(min(y), max(y), 100)

    # Bin the data into a 2D histogram
    [counts, x_bins, y_bins] = np.histogram2d(x, y, bins=[x_bins, y_bins])
    counts = np.transpose(counts)

    # Set up colormap scale
    # NOTE: Enforce 'linear' scale if counts are too low
    if ( np.max(counts) < 10 ):
        scale = 'linear'
    color_norm = None
    if ( scale == 'log' ):
        color_norm = colors.LogNorm(vmin=1, vmax=counts.max())
        # If using log-scale, set 0-count bins to 1. Otherwise we'll have missing pixels in the heatmap
        mask = counts == 0
        counts[mask] = 1

    # Render a heatmap from the counts, using bin limits to define the extent of the figure
    fig = plt.figure(figsize=[12.5,8], facecolor='w')
    ax = plt.gca()
    cb = plt.imshow(counts, cmap=cmap, norm=color_norm,
                    origin='lower', aspect=aspect, interpolation='bilinear',
                    extent=[x_bins[0],x_bins[-1],y_bins[0],y_bins[-1]])

    # Set up plot properties
    plt.title(title, size=24)
    plt.xlabel(xlabel, size=20)
    plt.ylabel(ylabel, size=20)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.grid('minor')
    fig.colorbar(cb, ax=ax)
    bbox_props = dict(facecolor='w', edgecolor='k')

    # (OPTIONAL) Plot a dashed line along the diagonal
    if ( plot_diagonal ):
        xlims = ax.get_xlim()
        ylims = ax.get_ylim()
        ax.plot(xlims, ylims, linestyle='--', linewidth=1, color=[0.25,0.25,0.25])

    # (OPTIONAL) Save figure if desired
    if ( output_filepath is not None ):
        plt.savefig(output_filepath, bbox_inches='tight')
    plt.show()

#
def plot_regression_accuracy(y_meas, y_pred,
                             min_error=0.0,
                             max_error=100.0,
                             error_type='relative',
                             title='Model Accuracy',
                             xlabel='Maximum Error\n(%)',
                             ylabel='Accuracy\n(%)',
                             output_filepath=None):
    '''Plot classification accuracy of a regression model as a function of maximum
       allowed error. For example, if we require predictions to be within +/- 10% of
       the true measured answer, then we can convert continuous regression output to
       a simple Y/N answer. Was the prediction within 10%?

       Calculates accuracy for increasingly permissive errors. A highly-performant
       model will have most samples fall within a small regression error.'''

    # Calculate accuracy as a function of regression error
    n_bins = 100
    error_limits = np.linspace(min_error, max_error, n_bins)
    accuracies = np.zeros(n_bins)
    for n in range(n_bins):
        accuracies[n] = regression_accuracy(y_meas, y_pred, max_error=error_limits[n], error_type=error_type)

    # Set up plot
    fig = plt.figure(figsize=[8,8], facecolor='w')
    ax = plt.gca()
    plt.plot(error_limits, 100*accuracies)

    # Set up plot properties
    plt.title(title, size=24)
    plt.xlabel(xlabel, size=20)
    plt.ylabel(ylabel, size=20)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.xlim([error_limits[0], error_limits[-1]])
    plt.ylim([0,100])
    plt.grid('minor')

    # (OPTIONAL) Save figure if desired
    if ( output_filepath is not None ):
        plt.savefig(output_filepath, bbox_inches='tight')
    plt.show()


## Define function to Load The data

In [146]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)
output_path = '/content'

## Define function to load dataset

In [147]:
def load_dataset(input_filepath, dataset_name=None, datetime_cols=None):
    '''Loads a *.txt file into a Pandas DataFrame'''
    
    t0 = time()
    df = pd.read_csv(input_filepath, sep=',', parse_dates=datetime_cols, infer_datetime_format=True)

    # Print status message
    print('Dataset Summary')
    print('----------------------------')
    print('Name:    {0:s}'.format(str(dataset_name)))
    print('Records: {0:d}'.format(df.shape[0]))
    print('Columns: {0:d}'.format(df.shape[1]))
    print('Size:    {0:3.2f}MB'.format(df.memory_usage(deep=True).sum() / 1000000))
    print('Time:    {0:3.2f}s'.format(time() - t0))
    print()
    
    return df

## Load the data

In [148]:
################################################################################
# Load input datasets
################################################################################

# Train dataset
input_filepath = '/content/training_dataset.csv'
train_df = load_dataset(input_filepath, dataset_name='train', datetime_cols=['observation_timestamp'])

# Test dataset
input_filepath = '/content/test_dataset.csv'
test_df = load_dataset(input_filepath, dataset_name='test', datetime_cols=['observation_timestamp'])


FileNotFoundError: ignored

## Split the training data into Training and Validation(test) dataset

In [ ]:
from sklearn.model_selection import train_test_split 
train_df,valid_df  = train_test_split(train_df,test_size=0.2,random_state=42)
print(f"train dataset shape :{train_df.shape}")
print(f"validation dataset shape : {valid_df.shape}")

## Defining Features and Targets

This is just for reference, competitors are free chose the features of their and experiment further.

In [ ]:
################################################################################
# Define training target & feature columns
# NOTE: All columns will need to be numeric to work with sklearn or XGBoost! 
################################################################################

# Target: What we're trying to predict
target_col = 'register__sales_dollar_amt_this_hour'
target_units = '$/hr'
target_label = 'Register Sales Per Hour'
target_range = [0,5000]

# Index: What uniquely determines each record?
index_col = 'observation_id'

# Features: What we're using to predict the target
# NOTE: See the data dictionary for a complete list of available features
feature_cols = [
    'store__n_open_registers', 
    'cashier__item_scan_rate_per_min',
    'register__peak_sales_dollar_amt_per_hour',
    'register__sales_quantity_rescanned_frac',
    'store__occupancy_outdoors',
    'region__n_open_registers',
    'region__peak_sales_dollar_amt_per_hour',
    #'region__peak_sales_dollar_amt_per_hour_v2',
    'hour_of_day'
]

# Metadata: Not used in model training, but kept to tag the data
metadata_cols = ['observation_timestamp']

# Unused: All other columns (e.x. Unusable or unknown columns)
available_cols = train_df.columns.values
used_cols = [index_col, target_col] + feature_cols + metadata_cols
unused_cols = [col for col in available_cols if col not in used_cols]

# Print summary stats for the target variable
target_mean = train_df[target_col].mean()
target_std = train_df[target_col].std()
target_min = train_df[target_col].min()
target_max = train_df[target_col].max()
target_quantiles = train_df[target_col].quantile([0.1, 0.25, 0.5, 0.75, 0.9]).values
target_iqr = target_quantiles[3] - target_quantiles[1]
print('Target Variable: {0:s}'.format(target_col))
print('----------------------------------------')
print('Units:  {0:s}'.format(target_units))
print('Mean:   {0:3.2f} +/- {1:3.2f}'.format(target_mean, target_std))
print('IQR:    {0:3.2f}'.format(target_iqr))
print()
print('|---------+---------+---------+---------+---------+---------+---------|')
print('|   Min   |   10%   |   25%   |   Med   |   75%   |   90%   |   Max   |')
print('|---------+---------+---------+---------+---------+---------+---------|')
print('|{0:^9.2f}|{1:^9.2f}|{2:^9.2f}|{3:^9.2f}|{4:^9.2f}|{5:^9.2f}|{6:^9.2f}|'.format(target_min, target_quantiles[0], target_quantiles[1], target_quantiles[2], target_quantiles[3], target_quantiles[4], target_max))
print('|---------+---------+---------+---------+---------+---------+---------|')


## Data Transformations

In [ ]:
################################################################################
# Do some basic transformations
################################################################################

# This will only include one-hot encoding & NULL-imputation here.
# Let's transform each input dataset separately
categorical_cols = ['register__payment_types_accepted', 'store__type_code']
imputation_cols = [col for col in feature_cols if col not in categorical_cols]

# Dataset: train
print('Transforming \'train\' dataset...')
train_df = encode_categoricals(train_df, categorical_cols)
[train_df, imputation_mapping] = impute_nulls(train_df, imputation_cols)
print()

# Dataset: test
print('Transforming \'test\' dataset...')
test_df = encode_categoricals(test_df, categorical_cols)
[test_df, imputation_mapping] = impute_nulls(test_df, imputation_cols, imputation_mapping=imputation_mapping)
print()

# Dataset: validation
print('Transforming \'validation\' dataset...')
valid_df = encode_categoricals(valid_df, categorical_cols)
[valid_df, imputation_mapping] = impute_nulls(valid_df, imputation_cols, imputation_mapping=imputation_mapping)
print()

# Print status
print()
print('Transformations complete.')


## Re-weight training samples

In [ ]:
################################################################################
# Reweight training samples to reduce bias & improve performance where we have
# less data available. The resulting model should generalize somewhat better
################################################################################

target_vals = train_df[target_col].values
target_bins = np.linspace(target_range[0],target_range[1],501)
smoothing_kernel_props = {'size': 51, 'sigma': 7}
sample_weights = get_sample_weights(target_vals, target_bins, 
                                    smoothing_kernel_props=smoothing_kernel_props, 
                                    weighting_method='inverse_square', 
                                    max_weight=10,
                                    make_plots=True)
train_df['sample_weights'] = sample_weights


## Build training & validation Datasests

In [ ]:
################################################################################
# Build train/test datasets that are suitable as modeling inputs
################################################################################

# Extract features, targets, and (optionally) weights
X_train = train_df[feature_cols].values
y_train = train_df[target_col].values
wt_train = train_df['sample_weights'].values

X_test = test_df[feature_cols].values
#y_test = test_df[target_col].values # Commented out as y_test does not exist in the data

X_valid = valid_df[feature_cols].values
y_valid = valid_df[target_col].values

print('Dataset Summary')
print('-'*40)
print('Train:         {0:>5d} samples  x  {1:d} features '.format(X_train.shape[0], X_train.shape[1]))
print('Test:          {0:>5d} samples  x  {1:d} features '.format(X_test.shape[0], X_test.shape[1]))
print('Validation:    {0:>5d} samples  x  {1:d} features '.format(X_valid.shape[0], X_valid.shape[1]))


## Build the XGBoost Model

In [ ]:
################################################################################
# Train an XGBoost model w/ hyperparameter tuning 
################################################################################

# Set up the ranges of hyperparameters to test
optimize_hyperparameters = False
n_runs = 250    # Number of pts to test during random search optimization
hyperparameter_space = {
    'n_estimators': [10,75],
    'learning_rate': [0.05,0.5],
    'max_depth': [1,4]
}
default_hyperparameters = {'n_estimators': 64, 'learning_rate': 0.2551, 'max_depth': 5}    # Used when tuning is skipped
metric_to_optimize = 'racc'    # Options: ['rmse', 'r2', 'mae', 'mape', 'racc']

# (OPTIONAL) Optimize hyperparameters via random search, if desired
if ( optimize_hyperparameters ):
    [xgb_model, hyperparameters, _] = random_search_xgb(hyperparameter_space, 
                                                        X_train, 
                                                        y_train, 
                                                        wt_train=wt_train,
                                                        n_runs=n_runs,
                                                        metric_to_optimize=metric_to_optimize)

# Otherwise, just train a model on the default parameters
else:
    hyperparameters = default_hyperparameters
    xgb_model = train_model_xgb(hyperparameters, X_train, y_train, wt_train=wt_train)
    

## Evaluate the performance of the model

In [ ]:
################################################################################
# Evaluate model performance ('validation' dataset)
# NOTE: Make sure 'valid_df' is already loaded & the data has been transformed!
################################################################################

# Evaluate model performance on both train/validation datasets
[eval_results__train, eval_results__test] = evaluate_regression_model(xgb_model, 
                                                                      X_train=X_train, y_train=y_train, wt_train=wt_train, 
                                                                      X_test=X_valid, y_test=y_valid, 
                                                                      y_lims=[0,None])
y_meas = eval_results__test['data']['y_meas']
y_pred = eval_results__test['data']['y_pred']
r2 = eval_results__test['r2']
result_df = pd.DataFrame({'prediction':y_pred})
print(result_df.head())
result_df.to_csv('validation_results.csv', index=False)
# Print a performance summary
n_samples__train = X_train.shape[0]
n_samples__test = X_valid.shape[0]
print('Model Performance Summary')
print('Target: {0:s} ({1:s})'.format(target_label, target_units))
print('-'*40)
print('Hyperparameters')
print('-- {0:<16s} {1:d}'.format('n_estimators:', hyperparameters['n_estimators']))
print('-- {0:<16s} {1:d}'.format('max_depth:', hyperparameters['max_depth']))
print('-- {0:<16s} {1:3.3f}'.format('learning_rate:', hyperparameters['learning_rate']))
print('\nModel Performance (Train)')
print('# of Samples: {0:d}'.format(n_samples__train))
print('-- R^2:  {0:>8.4f} +/- {1:3.4f}'.format(eval_results__train['r2']['mean'], eval_results__train['r2']['std']))
print('-- RMSE: {0:>8.2f} +/- {1:3.2f}'.format(eval_results__train['rmse']['mean'], eval_results__train['rmse']['std']))
print('-- MAE:  {0:>8.2f} +/- {1:3.2f}'.format(eval_results__train['mae']['mean'], eval_results__train['mae']['std']))
print('-- MAPE: {0:>8.1f} +/- {1:3.1f}%'.format(100*eval_results__train['mape']['mean'], 100*eval_results__train['mape']['std']))
print('-- RAcc relative: {0:>8.2f} +/- {1:3.2f}%'.format(100*eval_results__train['racc']['mean'], 100*eval_results__train['racc']['std']))
print('\nModel Performance (Test)')
print('# of Samples: {0:d}'.format(n_samples__test))
print('-- R^2:  {0:>8.4f}'.format(eval_results__test['r2']))
print('-- RMSE: {0:>8.2f}'.format(eval_results__test['rmse']))
print('-- MAE:  {0:>8.2f}'.format(eval_results__test['mae']))
print('-- MAPE: {0:>8.1f}%'.format(100*eval_results__test['mape']))
print('-- RAcc relative : {0:>8.2f}%'.format(100*eval_results__test['racc']))
print()

# Heatmap: Predicted vs. measured values
n_bins = 100
bins = np.linspace(target_range[0], target_range[1], n_bins)
title = 'DPhi Base Model - Heatmap\nValidation Set ($R^2$ = {0:3.2f})'.format(r2)
xlabel = 'Measured {0:s}\n({1:s})'.format(target_label, target_units)
ylabel = 'Predicted {0:s}\n({1:s})'.format(target_label, target_units)
output_filepath = '{0:s}/predicted_vs_measured__heatmap__validation_dataset.png'.format(output_path)
plot_heatmap(y_meas, y_pred,
             title=title, xlabel=xlabel, ylabel=ylabel,
             x_bins=bins, y_bins=bins, plot_diagonal=True, 
             output_filepath=output_filepath)

# Accuracy: Predicted vs. measured values
title = 'DPhi Base Model - Accuracy\nValidation Set ($R^2$ = {0:3.2f})'.format(r2)
xlabel = 'Maximum Error\n({0:s})'.format('%')
ylabel = 'Percentage of Predictions\n(%)'
output_filepath = '{0:s}/predicted_vs_measured__accuracy__validation_dataset.png'.format(output_path)
plot_regression_accuracy(y_meas, y_pred, max_error=25, error_type='relative', 
                         title=title, ylabel=ylabel, output_filepath=output_filepath)

# Distributions of Predicted & Measured Values
n_bins = 50
bins = np.linspace(target_range[0], target_range[1], n_bins)
data_labels = ['Measured', 'Predicted']
title = 'DPhi Base Model - Distribution\nValidation Set ($R^2$ = {0:3.2f})'.format(r2)
xlabel = '{0:s} ({1:s})'.format(target_label, target_units)
output_filepath = '{0:s}/predicted_vs_measured__distribution__validation_dataset.png'.format(output_path)
plot_distribution([y_meas, y_pred], data_labels=data_labels, bins=bins, 
                  title=title, xlabel=xlabel, hist_units='probability', 
                  output_filepath=output_filepath)


## Make Predictions

In [ ]:
X_test.shape

In [ ]:
predictions = xgb_model.predict(X_test)

In [ ]:
print(predictions)

## Export predictions into a .csv file for submission

In [ ]:
results_df = pd.DataFrame({'prediction':predictions})

In [ ]:
results_df.head()

In [ ]:
results_df.to_csv('predictions.csv',index=False)